<h1><center>Intuitive Explanation of Non-stationary Gaussian Process Kernels</center></h1>

## Background

### What are GPs

Gaussian Processes (GP) are powerful supervised learning methods, designed to solve classification and regression problems. One of the major advantages of using Gaussian processes is that it can estimate the uncertainty of its predictions by describing the probability distributions of the potentially infinite functions that fit the data. It can be defined as a stochastic process of random variables following a Gaussian distribution with a mean and a covariance function.

Görtler, et al., provides <a style="text-decoration:none" href="https://distill.pub/2019/visual-exploration-gaussian-processes/">an excellent visual exploration of Gaussian Processes</a> with mathematical intuition as well as a deeper understanding of how they work. In this article, we will cover some basic minimal concepts that help us set up a foundation for understanding Gaussian processes and extend it to assess its performance over regression problems.

### Kernels

A kernel (also called the covariance function) describes covariance, i.e, the joint variability of the Gaussian process random variables and is essential in setting up prior information on the GP distribution. These covariance functions make the core of the GP models. Radial basis function (RBF) kernel (also known as Gaussian kernel) is a popularly used covariance function in GP modelling.

\begin{align}
K_{rbf}(\mathbf{x}_i, \mathbf{x}_j) &= \sigma^2 \exp\left(\frac{||\mathbf{x}_i - \mathbf{x}_j||_2^2}{2l^2}\right)\\
\end{align}

There are a variety of kernels that can be used to model different desired shapes of the fitting functions. We also discuss two broad categories of kernels, stationary and non-stationary in Section ??, and also compare their performances on standard datasets.  Following parameters in the kernel function play a significant role in the modelling of a GP:

* Lengthscale ($l$)
* Variance ($\sigma^2$)

In case of regression problems, these parameters are learnt using the training data by minimizing the following negative log marginal likelihood (nlml) function.

\begin{align}
\log p(\mathbf{y}|X) &= -\frac{1}{2}y^T(K+\sigma^2_n I)^{-1}\mathbf{y} - \frac{1}{2}\log|K+\sigma^2_n I|-\frac{n}{2}\log2\pi\\
K &= \text{covariance_function}(X, X)\\
\sigma_n^2 &= \text{likelihood noise variance}\\
n &= \text{cardinality of } X \text{ or } \mathbf{y}
\end{align}

Now, let us visualize standatd (stationary) GPs applied on some standard datasets

## Stationary GP on noisy sine curve dataset

In [28]:
import GPy
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
# !pip install chart-studio

In [29]:
def get_fig():
    layout = go.Layout(
        paper_bgcolor='rgb(255,255,255)',
        plot_bgcolor='rgb(255,255,255)'
    )
    fig = go.Figure(layout=layout)
    return fig

In [42]:
np.random.seed(0)
X1 = np.linspace(-1, 1, 50).reshape(-1,1)
y1 = np.sin(5*X1) + np.random.rand(50, 1)*0.5
X1_ = np.linspace(-1, 1, 100).reshape(-1,1)
# X1.shape, y.shape

In [56]:
model = GPy.models.GPRegression(X1, y1, GPy.kern.RBF(1))
model.optimize_restarts(5, verbose=0)

y1_, y1_var = model.predict(X1_)
y1_std2 = np.sqrt(y1_var)*2

fig = get_fig()
fig.add_trace(go.Scatter(x=X1.ravel(), y=y1.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'))
fig.add_trace(go.Scatter(x=X1_.ravel(), y=y1_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'))
fig.add_trace(go.Scatter(x=X1_.ravel(), y=y1_.ravel()-y1_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ))
fig.add_trace(go.Scatter(x=X1_.ravel(), y=y1_.ravel()+y1_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'))
fig.update_layout(
    title_text='Gaussian process fit on noisy sine curve dataset with RBF kernel', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y"
                #font=dict(family="Courier New")
                )
fig.show()